In [1]:
import datafaucet as dfc

In [2]:
# start the engine
project = dfc.project.load()

created SparkEngine
Init engine "spark"
Connecting to spark master: local[*]
Engine context spark:2.4.4 successfully started


In [3]:
from pyspark.sql import functions as F
spark = dfc.context()

In [4]:
SIZE = 10_000_000

In [5]:
df = dfc.range(SIZE)

In [6]:
df = (df
    .cols.create('g').randint(1,32)
    .cols.create('v').randint(0,SIZE)
)

In [7]:
df.data.grid(5)

,id,g,v
0,0,8,8507490
1,1,10,8655161
2,2,11,1304930
3,3,24,8996440
4,4,26,8708332


In [8]:
(df
    .cols.get('v').agg('distinct')
).data.grid()

,v
0,6319560


In [9]:
(df
    .cols.get('v').agg(F.approx_count_distinct)
).data.grid()

,v
0,6128737


In [12]:
(df.cols
        .get('v')
        .groupby('g')
        .agg([F.countDistinct, F.approx_count_distinct])
).rows.sort('g').data.grid()

AttributeError: 'Rows' object has no attribute 'sort'

In [11]:
# using hll_init (less efficient, might OOM on large collections)

from datafaucet.spark import aggregations as A
from datafaucet.spark import functions 
(df
    .cols.get('v').hll_init()
    .cols.get('v').groupby('g').agg(A.hll_merge())
    .cols.get('v').hll_count() 
).data.grid()

Py4JJavaError: An error occurred while calling o381.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 24.0 failed 1 times, most recent failure: Lost task 1.0 in stage 24.0 (TID 1475, localhost, executor driver): org.apache.arrow.memory.OutOfMemoryException: Failure allocating buffer.
	at io.netty.buffer.PooledByteBufAllocatorL.allocate(PooledByteBufAllocatorL.java:57)
	at org.apache.arrow.memory.AllocationManager.<init>(AllocationManager.java:93)
	at org.apache.arrow.memory.BaseAllocator.bufferWithoutReservation(BaseAllocator.java:314)
	at org.apache.arrow.memory.BaseAllocator.buffer(BaseAllocator.java:283)
	at org.apache.arrow.memory.BaseAllocator.buffer(BaseAllocator.java:248)
	at org.apache.arrow.vector.BaseVariableWidthVector.reallocDataBuffer(BaseVariableWidthVector.java:497)
	at org.apache.arrow.vector.BaseVariableWidthVector.handleSafe(BaseVariableWidthVector.java:1225)
	at org.apache.arrow.vector.BaseVariableWidthVector.setSaf

In [13]:
# using hll_init_agg (very efficient: sketch in memory)

from datafaucet.spark import aggregations as A
(df
    .cols.get('v').groupby('g').agg(A.hll_init_agg())
    .cols.get('v').hll_count()
    .rows.sort('g')
).data.grid()

AttributeError: 'Rows' object has no attribute 'sort'

In [14]:
# using hll_init_agg (very efficient: sketch in memory)

from datafaucet.spark.aggregations import hll_init_agg
from datafaucet.spark.functions import hll_count
(df
    .groupby('g').agg(hll_init_agg()(F.col('v')).alias('v'))
    .select('g', hll_count()(F.col('v')).alias('count'))
).data.grid()

,g,count
0,31,315304
1,28,315569
2,26,328013
3,27,314135
4,12,317926
5,22,320043
6,1,319179
7,13,313253
8,6,320845
9,16,319876


In [35]:
#pre-aggregate the cube
df_cube = (df
    .cols.get('v').groupby('g').agg(('v_sketch',A.hll_init_agg()))
)
cube = df_cube.data.collect()
cube

In [16]:
# re-aqggregate on the cube with hll_merge + hll_count
(df_cube
    .cols.get('v').hll_count()
).data.grid()

,g,v
0,31,315304
1,28,315569
2,26,328013
3,27,314135
4,12,317926
5,22,320043
6,1,319179
7,13,313253
8,6,320845
9,16,319876


In [25]:
# partial count on the cube: filter + hll_merge + hll_count
(df_cube
     .rows.filter("g in(0,2,4)")
     .cols.get('v').agg(A.hll_merge())
     .cols.get('v').hll_count()
).data.grid()

,v
0,614438


In [26]:
# verify the result with countDistinct, and approx_count_distinct
(df
    .rows.filter("g in (0,2,4)")
    .cols.get('v').agg([F.countDistinct, F.approx_count_distinct])
).data.grid()

,v_countDistinct,v_approx_count_distinct
0,624766,610508


In [90]:
r = range(1,32)

sel = []
# last 7 days,
#sel += [[x for x in r if x> (31-7)]]
# by day of the week
sel += [[x for x in r if x%7==i] for i in range(7)]
# by week in the month
sel += [[x for x in r if x//7==i] for i in range(5)]
# by first, last half in the month
sel += [[x for x in r if x//16==i] for i in range(2)]
sel

[[7, 14, 21, 28],
 [1, 8, 15, 22, 29],
 [2, 9, 16, 23, 30],
 [3, 10, 17, 24, 31],
 [4, 11, 18, 25],
 [5, 12, 19, 26],
 [6, 13, 20, 27],
 [1, 2, 3, 4, 5, 6],
 [7, 8, 9, 10, 11, 12, 13],
 [14, 15, 16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25, 26, 27],
 [28, 29, 30, 31],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]]

In [91]:
s = [ str(x)[1:-1] for x in sel]
s

['7, 14, 21, 28',
 '1, 8, 15, 22, 29',
 '2, 9, 16, 23, 30',
 '3, 10, 17, 24, 31',
 '4, 11, 18, 25',
 '5, 12, 19, 26',
 '6, 13, 20, 27',
 '1, 2, 3, 4, 5, 6',
 '7, 8, 9, 10, 11, 12, 13',
 '14, 15, 16, 17, 18, 19, 20',
 '21, 22, 23, 24, 25, 26, 27',
 '28, 29, 30, 31',
 '1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15',
 '16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31']

In [92]:
%%time
def count_selection(e):
    return (df_cube
     .rows.filter(f"g in({e})")
     .cols.get('v').agg(A.hll_merge())
     .cols.get('v').hll_count()
    ).data.collect(1)['v'][0]

import functools

res =  [count_selection(e) for e in s]
spark_hll = pd.DataFrame(res, columns=['v_spark_hll'])
spark_hll

CPU times: user 454 ms, sys: 131 ms, total: 585 ms
Wall time: 57.1 s


,v_spark_hll
0,1204316
1,1441375
2,1497966
3,1488010
4,1195586
5,1236127
6,1193972
7,1729128
8,2048867
9,2002653


In [93]:
%%time
import pandas as pd
from HLL import HyperLogLog

def count_selection(e):
    hll_res = HyperLogLog(k=12)
    hll = HyperLogLog(k=12)
    for i in e:
        hll.set_registers(cube[cube['g']==i]['v'].iloc[0])
        hll_res.merge(hll)
    return int(hll_res.cardinality())

res =  [count_selection(e) for e in sel]
pandas_hll = pd.DataFrame(res, columns=['v_pandas_hll'])
pandas_hll

CPU times: user 93.2 ms, sys: 0 ns, total: 93.2 ms
Wall time: 92.6 ms


,v_pandas_hll
0,1204316
1,1441375
2,1497966
3,1488010
4,1195586
5,1236127
6,1193972
7,1729128
8,2048867
9,2002653


In [94]:
%%time
def count_selection(e):
    return (df
     .rows.filter(f"g in({e})")
     .cols.get('v').agg(F.countDistinct)
    ).data.collect(1)['v'][0]

res =  [count_selection(e) for e in s]
spark_distinct = pd.DataFrame(res, columns=['v_spark_distinct'])

CPU times: user 131 ms, sys: 26.3 ms, total: 158 ms
Wall time: 12.4 s


In [95]:
d = spark_distinct.join(spark_hll).join(pandas_hll)
d['error_pct'] = 100*(d['v_spark_hll']-d['v_spark_distinct'])/d['v_spark_distinct']
d

,v_spark_distinct,v_spark_hll,v_pandas_hll,error_pct
0,1210892,1204316,1204316,-0.543071
1,1488812,1441375,1441375,-3.186232
2,1490592,1497966,1497966,0.494703
3,1491201,1488010,1488010,-0.213989
4,1210231,1195586,1195586,-1.210100
5,1210485,1236127,1236127,2.118324
6,1209081,1193972,1193972,-1.249627
7,1759457,1729128,1729128,-1.723770
8,2020635,2048867,2048867,1.397185
9,2022932,2002653,2002653,-1.002456
